# Some Demonstrations with Minikube and Containers

## Cleaning Up

First, the traces of former executions of  *minikube* are eliminated, the cluster itself (stop, delete) and further initialization steps in the cache directory to establish a clean base.

In [1]:
!minikube stop ; minikube delete ; rm -rf ~/.minikube/cache

✋  Stopping node "minikube"  ...
🛑  Powering off "minikube" via SSH ...
🛑  1 nodes stopped.
🔥  Deleting "minikube" in docker ...
🔥  Deleting container "minikube" ...
🔥  Removing /Users/lothar/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.


Second, formerly built podman images will be deleted as well.

In [2]:
!podman rmi $(podman images -qa)! --force

Untagged: localhost/helloworld:v0.3.0
Untagged: localhost/helloworld:v0.2.0
Untagged: localhost/helloworld:v0.1.0
Untagged: docker.io/messense/rust-musl-cross:x86_64-musl
Untagged: docker.io/library/rust:1.55.0-alpine3.14
Untagged: docker.io/library/rust:1.55.0
Deleted: 09cc868a05f2336d638744f931b002c2bbc6e58089967cb0a5229f113e7dea2a
Deleted: 5fe1f5eb5a457d96a258092da6d5dafb480e054da443bbca4aa9c55ee919faac
Deleted: 4eff59f79ae6c88c754d2e93cabb83813d2206aeb0bd45c63e525757affdef0e
Deleted: aa78994a7a1e2fdb5ae4635ef2a413db350796a42c8897f4868af715122d6386
Deleted: 322bce99c3fdeb4cb8c20692865e62be7ace741d72151448d7f12aed4e7face7
Deleted: c0ff6c87f7dd4dd760ea8c9a488b7c8a289aefad7ab57d1e1bc5f85f0c269761
Deleted: 75df71096f49894d34f5d45138ed59fbc8b3107976e604f4dfb00fe2667c7c7b
Deleted: bbe0a33bde60a3bab2f970ba5c7c09256055eb78fbfec58be4c6df5cb5fd38b3
Deleted: d7c2293f0aef6be9689187bcaf2852d1d5a1d28c473eff47f8eb78153520017b
Deleted: 1e8b5c8405823ab415fd2478f85c2f6a28d444ec4b4730302913ef9a89c349d

'Error: image name or ID must be specified' indicates that the cache has bees empty before.

In [3]:
!rm -rf alpine

In [4]:
!rm -rf helloworld

## Start The First Cluster

In [5]:
%%time

!KUBERNETES_VERSION=1.22.1 minikube start --kubernetes-version=${KUBERNETES_VERSION}

😄  minikube v1.23.0 on Darwin 11.5.2
✨  Automatically selected the docker driver. Other choices: hyperkit, virtualbox, ssh, podman (experimental)
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
💾  Downloading Kubernetes v1.22.1 preload ...
    > preloaded-images-k8s-v12-v1...: 515.04 MiB / 515.04 MiB  100.00% 5.57 MiB
🔥  Creating docker container (CPUs=2, Memory=1987MB) ...
🐳  Preparing Kubernetes v1.22.1 on Docker 20.10.8 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default
CPU times: user 2.49 s, sys: 814 ms, total: 3.31 s
Wall time: 2min 21s


## Checking Readiness

In [6]:
!kubectl get nodes

NAME       STATUS     ROLES                  AGE   VERSION
minikube   NotReady   control-plane,master   11s   v1.22.1


In [7]:
!sleep 10 ; kubectl get nodes

NAME       STATUS   ROLES                  AGE   VERSION
minikube   Ready    control-plane,master   22s   v1.22.1


## Start Cluster With Preloaded Cache

In [8]:
!minikube stop ; minikube delete

✋  Stopping node "minikube"  ...
🛑  Powering off "minikube" via SSH ...
🛑  1 nodes stopped.
🔥  Deleting "minikube" in docker ...
🔥  Deleting container "minikube" ...
🔥  Removing /Users/lothar/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.


In [9]:
%%time

!KUBERNETES_VERSION=1.22.1 minikube start --kubernetes-version=${KUBERNETES_VERSION}

😄  minikube v1.23.0 on Darwin 11.5.2
✨  Automatically selected the docker driver. Other choices: hyperkit, virtualbox, ssh, podman (experimental)
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔥  Creating docker container (CPUs=2, Memory=1987MB) ...
🐳  Preparing Kubernetes v1.22.1 on Docker 20.10.8 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default
CPU times: user 663 ms, sys: 224 ms, total: 886 ms
Wall time: 39.9 s


Having deleted the cluster the startuo time for a new cluster with prefilled caches reduces to well under a minute.

In [10]:
!sleep 10

## Creating First Namespace And Pod

In [11]:
%%bash

cat <<EOF| kubectl apply -f -
kind: Namespace
apiVersion: v1
metadata:
  name: demo
  labels:
    name: demo
---
kind: Pod
apiVersion: v1
metadata:
  name: demo-pod
  namespace: demo
spec:
  containers:
  - name: demo-ctr
    image: nginx:stable-alpine
    resources:
      limits:
        memory: "200Mi"
        cpu: "700m"
      requests:
        memory: "200Mi"
        cpu: "700m"
EOF

namespace/demo created
pod/demo-pod created


A namespace demo with a pod demo-pod in it according to above yaml.

In [12]:
%%bash

cat <<EOF| kubectl apply -f -
kind: Namespace
apiVersion: v1
metadata:
  name: demo
  labels:
    name: demo
---
kind: Pod
apiVersion: v1
metadata:
  name: demo-pod
  namespace: demo
spec:
  containers:
  - name: demo-ctr
    image: nginx:stable-alpine
    resources:
      limits:
        memory: "200Mi"
        cpu: "700m"
      requests:
        memory: "200Mi"
        cpu: "700m"
EOF

namespace/demo unchanged
pod/demo-pod unchanged


As nemspace *demo* and the pod *demo-pod* in it have already been created by executing the exact same command before, *kubectl apply* remains idempotent.

In [13]:
!kubectl get pods --namespace demo

NAME       READY   STATUS              RESTARTS   AGE
demo-pod   0/1     ContainerCreating   0          0s


The newly created and started pod *demo-pod* is just one pod among others constituting the base kubernetes cluster. Those containers are isolated in the *kube-system* namespace.

In [14]:
!kubectl get namespaces

NAME              STATUS   AGE
default           Active   16s
demo              Active   1s
kube-node-lease   Active   18s
kube-public       Active   18s
kube-system       Active   18s


In [15]:
!kubectl get pods --namespace kube-system

NAME                               READY   STATUS              RESTARTS   AGE
coredns-78fcd69978-b4b9h           0/1     ContainerCreating   0          2s
etcd-minikube                      1/1     Running             0          13s
kube-apiserver-minikube            1/1     Running             0          13s
kube-controller-manager-minikube   1/1     Running             0          13s
kube-proxy-sc77r                   1/1     Running             0          2s
kube-scheduler-minikube            1/1     Running             0          16s
storage-provisioner                0/1     ContainerCreating   0          12s


In [16]:
%%bash

echo "systemd-cgls --no-pager ; exit" | minikube ssh

systemd-cgls --no-pager ; exit
]0;docker@minikube: ~docker@minikube:~$ systemd-cgls --no-pager ; exit
Control group /docker/72f6c802458aae83498117481d8b770ce511183a63957a627341d63c2b08cf6d:
-.slice
├─init.scope 
│ └─1 /sbin/init
└─system.slice 
  ├─containerd.service 
  │ ├─ 198 /usr/bin/containerd
  │ ├─1605 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 38e6043466da814…
  │ ├─1606 /usr/bin/containerd-shim-runc-v2 -namespace moby -id de061d1f6733959…
  │ ├─1607 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 341c171872cfd56…
  │ ├─1664 /pause
  │ ├─1665 /pause
  │ ├─1677 /pause
  │ ├─1697 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 88de5ec95b3370e…
  │ ├─1742 /pause
  │ ├─1789 /usr/bin/containerd-shim-runc-v2 -namespace moby -id bf3f5cf69be42db…
  │ ├─1808 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 67ee6fac6afd5e9…
  │ ├─1829 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 3a0016c96dcca87…
  │ ├─1853 /usr/bin/containerd-shim-runc-v2 -namespace moby

In [17]:
!skopeo --override-os linux copy docker://alpine:3.14.2 oci:alpine:3.14.2

Getting image source signatures
Copying blob a0d0a0d46f8b [====>-----------------------------] 403.2KiB / 2.7MiB
Copying blob a0d0a0d46f8b [======>---------------------------] 563.2KiB / 2.7MiB
Copying blob a0d0a0d46f8b [=============>----------------------] 1.1MiB / 2.7MiB
Copying blob a0d0a0d46f8b [================>-------------------] 1.3MiB / 2.7MiB
Copying blob a0d0a0d46f8b [=========================>----------] 1.9MiB / 2.7MiB
Copying blob a0d0a0d46f8b [===============================>----] 2.3MiB / 2.7MiB
Copying blob a0d0a0d46f8b [====================================] 2.7MiB / 2.7MiB
Copying blob a0d0a0d46f8b done  
Copying blob a0d0a0d46f8b done  
Copying blob a0d0a0d46f8b done  
Copying config 696d33ca15 [====================================] 585.0b / 585.0b
Copying config 696d33ca15 done  
Copying config 696d33ca15 done  
Copying config 696d33ca15 done  
Writing manifest to image destination
Storing signatures


In [18]:
!tree alpine

alpine
├── blobs
│   └── sha256
│       ├── 03014f0323753134bf6399ffbe26dcd75e89c6a7429adfab392d64706649f07b
│       ├── 696d33ca1510966c426bdcc0daf05f75990d68c4eb820f615edccf7b971935e7
│       └── a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e
├── index.json
└── oci-layout

2 directories, 5 files


In [19]:
!file alpine/blobs/sha256/*

alpine/blobs/sha256/03014f0323753134bf6399ffbe26dcd75e89c6a7429adfab392d64706649f07b: JSON data
alpine/blobs/sha256/696d33ca1510966c426bdcc0daf05f75990d68c4eb820f615edccf7b971935e7: JSON data
alpine/blobs/sha256/a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e: gzip compressed data, original size modulo 2^32 5865472


In [20]:
!cat alpine/blobs/sha256/03014f0323753134bf6399ffbe26dcd75e89c6a7429adfab392d64706649f07b | jq

{
  "schemaVersion": 2,
  "config": {
    "mediaType": "application/vnd.oci.image.config.v1+json",
    "digest": "sha256:696d33ca1510966c426bdcc0daf05f75990d68c4eb820f615edccf7b971935e7",
    "size": 585
  },
  "layers": [
    {
      "mediaType": "application/vnd.oci.image.layer.v1.tar+gzip",
      "digest": "sha256:a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e",
      "size": 2814446
    }
  ]
}


In [21]:
!cat alpine/blobs/sha256/696d33ca1510966c426bdcc0daf05f75990d68c4eb820f615edccf7b971935e7 | jq

{
  "created": "2021-08-27T17:19:45.758611523Z",
  "architecture": "amd64",
  "os": "linux",
  "config": {
    "Env": [
      "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
    ],
    "Cmd": [
      "/bin/sh"
    ]
  },
  "rootfs": {
    "type": "layers",
    "diff_ids": [
      "sha256:e2eb06d8af8218cfec8210147357a68b7e13f7c485b991c288c2d01dc228bb68"
    ]
  },
  "history": [
    {
      "created": "2021-08-27T17:19:45.553092363Z",
      "created_by": "/bin/sh -c #(nop) ADD file:aad4290d27580cc1a094ffaf98c3ca2fc5d699fe695dfb8e6e9fac20f1129450 in / "
    },
    {
      "created": "2021-08-27T17:19:45.758611523Z",
      "created_by": "/bin/sh -c #(nop)  CMD [\"/bin/sh\"]",
      "empty_layer": true
    }
  ]
}


In [22]:
!rustup self uninstall -y

info: removing rustup home
info: removing cargo home
info: removing rustup binaries
info: rustup is uninstalled


In [23]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-apple-darwin
info: syncing channel updates for 'stable-x86_64-apple-darwin'
info: latest update on 2021-09-09, rust version 1.55.0 (c8dfcfe04 2021-09-06)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
 17.1 MiB /  17.1 MiB (100 %)   6.0 MiB/s in  2s ETA:  0s
info: downloading component 'rust-std'
 21.0 MiB /  21.0 MiB (100 %)   6.5 MiB/s in  3s ETA:  0s
info: downloading component 'rustc'
 75.9 MiB /  75.9 MiB (100 %)   6.3 MiB/s in 11s ETA:  0s
info: downloading component 'rustfmt'
info: installing component 'cargo'
info: installing component 'clippy'
info: installing component 'rust-docs'
 17.1 MiB /  17.1 MiB (100 %)   5.7 MiB/s in  2s ETA:  0s
info: installing component 'rust-std'
 21.0 MiB /  21.0 MiB (100 %)  13.0 MiB/s in  1s ETA:  0s
info: installing component 'rustc'
 75.9 MiB /  75.9 MiB (100 %)  10.7 MiB/s in  6s ETA

In [24]:
!rustup default stable

info: using existing install for 'stable-x86_64-apple-darwin'
info: default toolchain set to 'stable-x86_64-apple-darwin'

  stable-x86_64-apple-darwin unchanged - rustc 1.55.0 (c8dfcfe04 2021-09-06)



In [25]:
!rustup update

info: syncing channel updates for 'stable-x86_64-apple-darwin'
info: checking for self-updates

  stable-x86_64-apple-darwin unchanged - rustc 1.55.0 (c8dfcfe04 2021-09-06)

info: cleaning up downloads & tmp directories


In [26]:
!cargo new helloworld

     Created binary (application) `helloworld` package


In [27]:
%%bash

cd helloworld

cat >Dockerfile <<EOF
FROM docker.io/library/rust:1.55.0

WORKDIR /usr/src/helloworld

COPY . .

RUN cargo build --release

RUN cargo install --path .

CMD ["/usr/local/cargo/bin/helloworld"]
EOF

In [28]:
%%bash

cd helloworld

cat >Dockerfile.slim <<EOF
FROM docker.io/library/rust:1.55.0-alpine3.14

WORKDIR /usr/src/helloworld

COPY . .

RUN cargo build --release

RUN cargo install --path .

CMD ["/usr/local/cargo/bin/helloworld"]
EOF

In [29]:
%%bash

cd helloworld

cat >Dockerfile.slim-staged <<EOF
# ------------------------------------------------------------------------------
# Cargo Build Stage
# ------------------------------------------------------------------------------

FROM docker.io/messense/rust-musl-cross:x86_64-musl as cargo-build

# RUN apt-get update

# RUN apt-get install musl-tools -y

RUN rustup target add x86_64-unknown-linux-musl 1>/dev/null 2>&1

WORKDIR /usr/src/helloworld

COPY . .

RUN cargo build --release --target=x86_64-unknown-linux-musl

# ------------------------------------------------------------------------------
# Final Stage
# ------------------------------------------------------------------------------

FROM docker.io/library/alpine:3.14.2

RUN addgroup -g 1000 helloworld

RUN adduser -D -s /bin/sh -u 1000 -G helloworld helloworld

WORKDIR /home/helloworld/bin/

COPY --from=cargo-build /usr/src/helloworld/target/x86_64-unknown-linux-musl/release/helloworld .

RUN chown helloworld:helloworld helloworld

USER helloworld

CMD ["./helloworld"]
EOF

In [30]:
!podman info

host:
  arch: amd64
  buildahVersion: 1.22.3
  cgroupControllers: []
  cgroupManager: systemd
  cgroupVersion: v2
  conmon:
    package: conmon-2.0.29-2.fc34.x86_64
    path: /usr/bin/conmon
    version: 'conmon version 2.0.29, commit: '
  cpus: 1
  distribution:
    distribution: fedora
    version: "34"
  eventLogger: journald
  hostname: localhost
  idMappings:
    gidmap:
    - container_id: 0
      host_id: 1000
      size: 1
    - container_id: 1
      host_id: 100000
      size: 65536
    uidmap:
    - container_id: 0
      host_id: 1000
      size: 1
    - container_id: 1
      host_id: 100000
      size: 65536
  kernel: 5.13.13-200.fc34.x86_64
  linkmode: dynamic
  memFree: 1724989440
  memTotal: 2061852672
  ociRuntime:
    name: crun
    package: crun-1.0-1.fc34.x86_64
    path: /usr/bin/crun
    version: |-
      crun version 1.0
      commit: 139dc6971e2f1d931af520188763e984d6cdfbf8
      spec: 1.0.0
      +SYSTEMD +SELINUX +APPARMOR +CAP +SECCOMP +EBPF +CRIU +YAJL
  os: l

In [31]:
!cd helloworld ; podman build --tag helloworld:v0.1.0 --file Dockerfile .

STEP 1/6: FROM docker.io/library/rust:1.55.0
Trying to pull docker.io/library/rust:1.55.0...
Getting image source signatures
Copying blob sha256:74ca45178655a5c9b4a372bb131e0e6a563074c81321f9178fd06bc9847f6559
Copying blob sha256:955615a668ce169f8a1443fc6b6e6215f43fe0babfb4790712a2d3171f34d366
Copying blob sha256:2756ef5f69a5190f4308619e0f446d95f5515eef4a814dbad0bcebbbbc7b25a8
Copying blob sha256:911ea9f2bd51e53a455297e0631e18a72a86d7e2c8e1807176e80f991bde5d64
Copying blob sha256:27b0a22ee906271a6ce9ddd1754fdd7d3b59078e0b57b6cc054c7ed7ac301587
Copying blob sha256:8584d51a9262f9a3a436dea09ba40fa50f85802018f9bd299eee1bf538481077
Copying blob sha256:27b0a22ee906271a6ce9ddd1754fdd7d3b59078e0b57b6cc054c7ed7ac301587
Copying blob sha256:955615a668ce169f8a1443fc6b6e6215f43fe0babfb4790712a2d3171f34d366
Copying blob sha256:8584d51a9262f9a3a436dea09ba40fa50f85802018f9bd299eee1bf538481077
Copying blob sha256:74ca45178655a5c9b4a372bb131e0e6a563074c81321f9178fd06bc9847f6559
Copying blob sha256:911ea

In [32]:
!podman images

REPOSITORY                TAG         IMAGE ID      CREATED       SIZE
localhost/helloworld      v0.1.0      1a6b3258ec3c  26 hours ago  1.28 GB
docker.io/library/rust    1.55.0      378164a392dd  4 days ago    1.27 GB
docker.io/library/alpine  3.14.2      14119a10abf4  2 weeks ago   5.87 MB


In [33]:
!podman run --name helloworld --rm helloworld:v0.1.0

Hello, world!


In [34]:
!cd helloworld ; podman build --tag helloworld:v0.2.0 --file Dockerfile.slim .

STEP 1/6: FROM docker.io/library/rust:1.55.0-alpine3.14
Trying to pull docker.io/library/rust:1.55.0-alpine3.14...
Getting image source signatures
Copying blob sha256:922454686f1ed186a1f1ecf76b947b3b16125d09227ba5d5736ce77c72941ce0
Copying blob sha256:a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e
Copying blob sha256:d4ea2bb3df1724da0bb9bbf8d43f7a5b7192610627c590c2fd4b8e0770230cbf
Copying blob sha256:922454686f1ed186a1f1ecf76b947b3b16125d09227ba5d5736ce77c72941ce0
Copying blob sha256:d4ea2bb3df1724da0bb9bbf8d43f7a5b7192610627c590c2fd4b8e0770230cbf
Copying config sha256:ac41586fb44e3aa14cee23647a0d7257d86f4a549634453ab95497bd24eff193
Writing manifest to image destination
Storing signatures
STEP 2/6: WORKDIR /usr/src/helloworld
--> 7d6c97951b4
STEP 3/6: COPY . .
--> 24bc0e51ddc
STEP 4/6: RUN cargo build --release
   Compiling helloworld v0.1.0 (/usr/src/helloworld)
    Finished release [optimized] target(s) in 1.16s
--> 2bf9c594385
STEP 5/6: RUN cargo install --path .
 

In [35]:
!cd helloworld ; podman build --tag helloworld:v0.3.0 --file Dockerfile.slim-staged .

[1/2] STEP 1/5: FROM docker.io/messense/rust-musl-cross:x86_64-musl AS cargo-build
Trying to pull docker.io/messense/rust-musl-cross:x86_64-musl...
Getting image source signatures
Copying blob sha256:6df3fa71a3d870b1a7b049f38596e2a5ecd1f439c2f596ea21740bd6176763f6
Copying blob sha256:35807b77a593c1147d13dc926a91dcc3015616ff7307cc30442c5a8e07546283
Copying blob sha256:7c0b836343229a8e7779760bf37b3e8a01cb469106b5294c4e33caa169ed146f
Copying blob sha256:f189444528f8e1bdef31e02d957842afa6399b8310a1ee8040ca41161fb303c8
Copying blob sha256:248be333cbffe89cd6668203381d5f882398ffb0a0f6b36905507703251d1f66
Copying blob sha256:3bcbd5f58845288bd66afa56ad7705428b5f16d4ee15486937e52bd5d3fb361b
Copying blob sha256:6df3fa71a3d870b1a7b049f38596e2a5ecd1f439c2f596ea21740bd6176763f6
Copying blob sha256:3bcbd5f58845288bd66afa56ad7705428b5f16d4ee15486937e52bd5d3fb361b
Copying blob sha256:248be333cbffe89cd6668203381d5f882398ffb0a0f6b36905507703251d1f66
Copying blob sha256:35807b77a593c1147d13dc926a91dcc3015

In [36]:
!podman images

REPOSITORY                          TAG                IMAGE ID      CREATED       SIZE
localhost/helloworld                v0.3.0             7476421a6811  26 hours ago  13.7 MB
<none>                              <none>             a720311b1653  26 hours ago  2.07 GB
localhost/helloworld                v0.2.0             a9bac2830c3f  26 hours ago  880 MB
localhost/helloworld                v0.1.0             1a6b3258ec3c  26 hours ago  1.28 GB
docker.io/messense/rust-musl-cross  x86_64-musl        b4f264478670  2 days ago    2.06 GB
docker.io/library/rust              1.55.0-alpine3.14  ac41586fb44e  4 days ago    868 MB
docker.io/library/rust              1.55.0             378164a392dd  4 days ago    1.27 GB
docker.io/library/alpine            3.14.2             14119a10abf4  2 weeks ago   5.87 MB


In [37]:
!podman images | grep helloworld | sort

localhost/helloworld                v0.1.0             1a6b3258ec3c  26 hours ago  1.28 GB
localhost/helloworld                v0.2.0             a9bac2830c3f  26 hours ago  880 MB
localhost/helloworld                v0.3.0             7476421a6811  26 hours ago  13.7 MB


In [38]:
!podman run --name helloworld --rm helloworld:v0.3.0

Hello, world!
